In [ ]:
import os
import sys
from datetime import datetime
import pandas as pd
from pandas import DataFrame
import numpy as np

import statistics
import random
from scipy import stats
import math
import statistics

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

import tensorflow as tf
from tensorflow.keras.models import load_model

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv1D, MaxPooling1D, AveragePooling, Input, Reshape, BatchNormalization, Concatenate

os.getcwd()
workdir = "your working directory"
datadir = workdir + "KinomeScan_data/"
resultdir = datadir + "input_result/"
predictiondir = datadir + "prediction_result/"
modeldir = workdir + "model/"

today = datetime.today().strftime("%Y%m%d")
today

In [ ]:
# load whole dataset from npz

with tf.device('/GPU:0'):
    train_data = np.load(resultdir + 'toy_train.npz')
    val_data = np.load(resultdir + 'toy_val.npz')
    test_data = np.load(resultdir + 'toy_test.npz')
    train_data.files, val_data.files, test_data.files

In [ ]:
x_train, y_train = train_data['x'], train_data['y']
x_val, y_val = val_data['x'], val_data['y']
x_test, y_test = test_data['x'], test_data['y']
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

In [ ]:
random.seed(2021)
LR = 0.000001
training_epochs = 200
batch_size = 100

In [ ]:
num_classes = 3
learning_rate = 0.0001
training_epochs = 100
batch_size = 25

In [ ]:
def rmse (y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred -y_true)))

In [ ]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# model building
with tf.device('/GPU:0'):

    model = Sequential()

## for yoon # Input size should be [batch, 1d, 2d, ch] = (None, 1, 15000, 1)
    model.add(Conv1D (kernel_size = 11, filters = 16, input_shape=(x_train.shape[1],x_train.shape[2]), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D (kernel_size = 11, filters = 16, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2, strides = 2, padding='same'))
    model.add(Conv1D (kernel_size = 11, filters = 32, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D (kernel_size = 11, filters = 32, padding='same'))
    model.add(MaxPooling1D(pool_size = 2, strides = 2, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv1D (kernel_size = 11, filters = 64, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2, strides = 2, padding='same'))
    model.add(Conv1D (kernel_size = 11, filters = 64, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2, strides = 2, padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense (1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense (2048))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense (4096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense (2048))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense (1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    # model.add(Dense(num_classes, activation = 'softmax',activity_regularizer=keras.regularizers.l2()  ))
    model.add(Dense(1, activation = 'linear', activity_regularizer= tf.keras.regularizers.L2()  ))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=LR), loss = rmse, metrics =["mse"])
    model.summary()

In [ ]:
model_train = model.fit(x_train, y_train, batch_size=32,epochs=100,verbose=1,
                        validation_data=(x_test, y_test))

In [ ]:
### RMSE plot
plt.figure()
plt.title("RMSE")
plt.plot(model_train.history['loss'],"r", label = "Tranning")
plt.plot(model_train.history['val_loss'], label = "Validation", color = 'dodgerblue')
plt.xlabel("Epochs")
plt.ylabel("RMSE")
plt.legend(loc=2)
# plt.savefig(model_name+"_"+timetail+"_rmse.png")

#Using when system memory is insufficient
#train_X=[]
#tval_X=[]

In [ ]:
pred_test = model.predict(x_test)
test_rmse = np.sqrt(mean_squared_error(y_test, pred_test))
test_r2 = r2_score(y_test, pred_test)
print("test RMSE : ", test_rmse)
print("test R2 : ", test_r2)

In [ ]:
# Draw scatter plot
plt.xlabel("Original",size = 20)
plt.ylabel("Predictions",size = 20)
plt.scatter(y_test, pred_test, s=5)
ident = [y_test.min(),y_test.max()]
plt.plot(ident,ident,'--', color='black',linewidth=2)

plt.show()

In [ ]:
# Option 2: Save/Load the Entire Model
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')